# Load packages

In [210]:
import kfp
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler

# Enter your gateway loadbalancer and the token from the cookie
[Use this extension on chrome to get token]( https://chrome.google.com/webstore/detail/editthiscookie/fngmhnnpilhplaeedifhccceomclgfbg?hl=en)

![image.png](./image.png)

In [211]:
#Update values for the load balancer and auth session
URL='<your-load-balancer-url>'
AUTH="authservice_session=<Key>"
namespace="admin"

In [122]:
client = kfp.Client(host=URL+"/pipeline", cookies=AUTH)
experiments = client.list_experiments(namespace=namespace)
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 2, 22, 10, 16, 13, tzinfo=tzlocal()),
 'description': None,
 'id': 'bc05a15c-5ac6-4cdf-935b-7a6451089d2c',
 'name': 'sample',
 'resource_references': [{'key': {'id': 'admin', 'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

### Using dsl for model archiver and torchserve integration
#### Init container is used for mar file generation

In [123]:
VOLUME_NAME = "pvcm"
DEPLOY="torchserve-bert"
MODEL="bert"

In [215]:
bert_data_prep_op = components.load_component_from_file(
    "./pytorch/data_prep/component.yaml"
)

bert_train_op = components.load_component_from_file(
    "./pytorch/train/component.yaml"
)

mar_op = load_component_from_file("./model_archive/component.yaml")
deploy_op = load_component_from_file("./deploy/component.yaml")

@dsl.pipeline(name="Training pipeline", description="Sample training job test")
def pytorch_bert():


    vop = dsl.VolumeOp(
        name=volume_name,
        resource_name=volume_name,
        modes=dsl.VOLUME_MODE_RWO,
        size="10Gi"
    )

    @dsl.component
    def download(url: str, output_path:str):
        return dsl.ContainerOp(
            name='Download',
            image='busybox:latest',
            command=["sh", "-c"],
            arguments=["mkdir -p %s; wget %s -P %s" % (output_path, url, output_path)],
        )

    @dsl.component
    def copy_contents(input_dir: str, output_dir:str):
        return dsl.ContainerOp(
            name='Copy',
            image='busybox:latest',
            command=["cp", "-R", "%s/." % input_dir, "%s" % output_dir],            
        )

    @dsl.component
    def mkdir(input_dir: str):
        return dsl.ContainerOp(
            name='mkdir',
            image='busybox:latest',
            command=["mkdir", "-p", "%s" % input_dir],            
        )

    @dsl.component
    def ls(input_dir: str):
        return dsl.ContainerOp(
            name='list',
            image='busybox:latest',
            command=["ls", "-R", "%s" % input_dir]
        )

    prep_output = bert_data_prep_op(
        input_data =
            [{"dataset_url":"https://kubeflow-dataset.s3.us-east-2.amazonaws.com/ag_news_csv.tar.gz"}],
        container_entrypoint = [
            "python",
            "/pvc/input/bert_pre_process.py",
        ],
        output_data = ["/pvc/output/processing"],
        source_code = ["https://kubeflow-dataset.s3.us-east-2.amazonaws.com/bert_pre_process.py"],
        source_code_path = ["/pvc/input"]
    ).add_pvolumes({"/pvc":vop.volume})

    train_output = bert_train_op(
        input_data = ["/pvc/output/processing"],
        container_entrypoint = [
            "python",
            "/pvc/input/bert_train.py",
        ],
        output_data = ["/pvc/output/train/models"],
        input_parameters = [{"tensorboard_root": "/pvc/output/train/tensorboard", 
        "max_epochs": 1, "num_samples": 150, "batch_size": 4, "num_workers": 1, "learning_rate": 0.001, 
        "accelerator": None}],
        source_code = ["https://kubeflow-dataset.s3.us-east-2.amazonaws.com/bert_datamodule.py", "https://kubeflow-dataset.s3.us-east-2.amazonaws.com/bert_train.py"],
        source_code_path = ["/pvc/input"]
    ).add_pvolumes({"/pvc":vop.volume}).after(prep_output)

    list_input = ls("/pvc/output").add_pvolumes({"/pvc":vop.volume}).after(train_output)

    properties = download(url='https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/bert/properties.json', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(vop)
    requirements = download(url='https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/bert/requirements.txt', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(vop)
    extrafile = download(url='https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/bert/index_to_name.json', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(vop)
    vocabfile = download(url='https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/bert/bert-base-uncased-vocab.txt', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(vop)
    handlerfile = download(url='https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/bert/bert_handler.py', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(vop)

    copy_files = copy_contents(input_dir="/pvc/output/train/models", output_dir="/pvc/input").add_pvolumes({"/pvc":vop.volume}).after(train_output)
    list_input = ls("/pvc/input").add_pvolumes({"/pvc":vop.volume}).after(copy_files)

    mar_task = mar_op(
        input_dir="/pvc/input",
        output_dir="/pvc/output",
        handlerfile="image_classifier").add_pvolumes({"/pvc":vop.volume}).after(list_input)

    list_output = ls("/pvc/output").add_pvolumes({"/pvc":vop.volume}).after(mar_task)

    production = deploy_op(
        action="apply",
        model_name="%s" % model_name,
        model_uri="pvc://{{workflow.name}}-%s/output" % volume_name,
        namespace="%s" % namespace,
        framework='pytorch'
    ).add_pvolumes({"/pvc":vop.volume}).after(list_output)
    
#   Below two lines are for canary deployment (we use same model for canary testing)
    mkdir_canary = mkdir(input_dir="/pvc/canary/output").add_pvolumes({"/pvc":vop.volume}).after(vop)
    canary_model = copy_contents(input_dir="/pvc/output", output_dir="/pvc/canary/output").add_pvolumes({"/pvc":vop.volume}).after(mar_task)

#   Canary Deployment
    canary = deploy_op(
        action="apply",
        model_name="%s" % model_name,
        model_uri="pvc://{{workflow.name}}-%s/canary/output" % volume_name,
        namespace="%s" % namespace,
        canary_traffic_percent="20",
        framework='pytorch'
    ).add_pvolumes({"/pvc":vop.volume}).after(canary_model)

In [216]:
# Compile pipeline
compiler.Compiler().compile(pytorch_bert, 'pytorch.tar.gz', type_check=True)

In [217]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-bert', 'pytorch.tar.gz')

### Wait for inference service below to go to `READY True` state.

In [204]:
!kubectl get isvc $DEPLOY

NAME              URL   READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                       AGE
torchserve-bert         False          100                              torchserve-bert-predictor-default-mdhs6   3m52s


# Inference

In [203]:
!kubectl get isvc $DEPLOY -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3

In [202]:
S_HOSTNAME=!kubectl get isvc $DEPLOY  -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3
SERVICE_HOSTNAME=S_HOSTNAME[0]
SERVICE_HOSTNAME

IndexError: list index out of range

# Prediction

In [199]:
!curl -v -H "Host: $SERVICE_HOSTNAME" -H "Cookie: $AUTH" "$URL/v1/models/$MODEL:predict" -d @./sample.txt

*   Trying 52.43.180.253...
* TCP_NODELAY set
* Connected to ad7fc227388c3431f9c778285d481334-127942271.us-west-2.elb.amazonaws.com (52.43.180.253) port 80 (#0)
> POST /v1/models/bert:predict HTTP/1.1
> Host: error: error executing jsonpath {.items[0].status.url}: Error executing template: array index out of bounds: index 0, length 0. Printing more information for debugging the template:
> User-Agent: curl/7.58.0
> Accept: */*
> Cookie: authservice_session=MTYxNTk3MDU1OHxOd3dBTkZGRVIxcFNOMUZEVGxGTlFVd3pXRUZRTkVOQ1JsbFROVUZGUkVjM1RGRXpTbGhaVVVwUVQxVTJSbFl6UWtoU1VrNUdUMUU9fEqY1QCbQAToNhQAqHwFfhh_pwhDDySKeKLlP8ttId77
> Content-Length: 84
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 84 out of 84 bytes
< HTTP/1.1 400 Bad Request
< date: Wed, 17 Mar 2021 11:59:34 GMT
< server: istio-envoy
< connection: close
< content-length: 0
< 
* Closing connection 0


# Check canary revisions
### Run below command in shell

In [ ]:
!kubectl get revisions -l serving.kubeflow.org/inferenceservice=$DEPLOY